# Формула расчёта смещения центров масс для ящика

### Исходные данные

In [1]:
# Характеристика 4-х осной ж/д платформы
L = 13400 # Длина пола
W = 2870 # Ширина пола
Qv = 21 # Масса тары
wugr = 1310 # Высота пола от УГР
height_ct_vagon = 800 # Высота центра тяжести(ЦТ) от УГР
9720 # База платформы

# delta = [12168, 0, 3651, 7752] # расстояние между грузами (нет в записке)
delta = [690, 3055, 6930, 10915]

# Вес - в тоннах.
# Длина, ширина, высота - в мм.
cargo = [ # грузы
    {'length': 1080, 'width': 1580, 'height': 390, 'weight': 395 / 1000},  
    {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6670 / 1000},
    {'length': 4100, 'width': 1720, 'height': 1150, 'weight': 1865 / 1000},
    {'length': 3870, 'width': 2890, 'height': 1020, 'weight': 4085 / 1000},
]
weightSum = sum(map(lambda x: x['weight'], cargo))

### Формулы

In [83]:
for index, item in enumerate(cargo):
    item['ct'] = delta[index]

# Продольное смещение грузов в вагоне - в мм.
Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) / weightSum)

# Продольное смещение грузов с вагоном - в мм.
Lcv = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) + Qv * L/2) / (weightSum + Qv)

# Общая высота ЦТ - в мм.
H = sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) / weightSum

print(Lc, Lcv, H)

694.4967345370733 265.7320299867706 1942.750672301191


In [71]:
cargo

[{'length': 1080, 'width': 1580, 'height': 390, 'weight': 0.395, 'ct': 1230.0},
 {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6.67, 'ct': 6650.0},
 {'length': 4100,
  'width': 1720,
  'height': 1150,
  'weight': 1.865,
  'ct': 17455.0},
 {'length': 3870,
  'width': 2890,
  'height': 1020,
  'weight': 4.085,
  'ct': 32355.0}]

### Доп формулы

In [8]:
# Общая высота ЦТ
height_center_gravity_in_car = (sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) + Qv * height_ct_vagon)/ (weightSum + Qv)
height_center_gravity_in_car

1237.2453329413495

In [10]:
# Расчет наветренной поверхности.
# Значение 7 - константа
sum([item['length'] / 1000 * item['height'] / 1000 for item in cargo]) + 7

21.5586

### Оптимизация

In [12]:
import random


# грузы
cargo = [
    {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6670 / 1000, 'delta_length': None, 'delta_width': None},
    {'length': 1080, 'width': 1580, 'height': 390, 'weight': 395 / 1000, 'delta_length': None, 'delta_width': None}, 
    {'length': 3870, 'width': 2890, 'height': 1020, 'weight': 4085 / 1000, 'delta_length': None, 'delta_width': None},
    {'length': 4100, 'width': 1720, 'height': 1150, 'weight': 1865 / 1000, 'delta_length': None, 'delta_width': None},
]
weightSum = sum(map(lambda x: x['weight'], cargo))

# Свободное место на платформе, если расположить все грузы.
free_L = L - sum([item['length'] for item in cargo])
print(f'{free_L = }')

# Даём пустое пространство на один груз, где можем его двигать.
free_L_for_one_cargo = free_L // len(cargo)
print(f'{free_L_for_one_cargo = }')

if free_L < 0:
    print('Груз не влазит!')

free_L = 700
free_L_for_one_cargo = 175


In [13]:
import itertools


class Optimize:
    """ Модель для оптимизации продольного смещения. """

    def __init__(self, cargo: list, step=1, n_iter=1000):
        """ step - шаг влево/вправо для оптимизации. Чем меньше, тем лучше. От 1 до бесконечности - int.
        n_iter - кол-во повторений алгоритма для надёжности. Чем болешье, чем точнее, но медленнее. """
        
        self.best_cargo = None
        self.score_length = float('inf')
        self.score_width = float('inf')

        self._step = step  # Чем меньше - тем лучше
        self._n_iter = n_iter  # Кол-во повторений
        
        self._optimize(cargo)


    def _init_delta(self, cargo: list):
        """ Инициализация значений delta. Внутри создаёт новый cargo. """
    
        # copy_cargo = copy.deepcopy(cargo)
        copy_cargo = [{k:v for k, v in dct.items()} for dct in cargo]

        # Единственный ящик инициализирую в центре по ширине и длине.
        if len(copy_cargo) == 1:
            copy_cargo[0]['delta_length'] = L / 2 - copy_cargo[0]['length'] / 2
            copy_cargo[0]['delta_width'] = W / 2 - copy_cargo[0]['width'] / 2
    
        else:
    
            prev_border = 0
            for idx, item in enumerate(copy_cargo):

                copy_cargo[idx]['delta_length'] = random.randint(prev_border, prev_border + free_L_for_one_cargo - 1)

                # Всегда начинаем с центра ширины.
                copy_cargo[idx]['delta_width'] = W / 2 - copy_cargo[idx]['width'] / 2
    
                # Обновляем левую границу.
                prev_border = prev_border + item['length'] + free_L_for_one_cargo

        
        return copy_cargo
    
    
    def _get_score_length(self, cargo: list):
        """ Получение метрик при движения груза продольно. """

        # Продольное смещение грузов в вагоне - в мм.
        Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['delta_length'], cargo)) / weightSum)
    
        return Lc

    def _get_score_width(self, cargo: list):
        """ Получение метрик при движения груза поперечно. """

        # Поперечное смещение грузов в вагоне - в мм.
        Bc = 0.5*W - (sum(map(lambda x: x['weight'] * x['delta_width'], cargo))) / weightSum
        return Bc
    
    def _check_best_cargo(self, cargo: list, length_flag: bool):
        """ Проверка метрик у обновлённого delta у cargo. """

        # Для продольного смещения.
        if length_flag:
            score = self._get_score_length(cargo)
            if abs(score) < abs(self.score_length):
                self.best_cargo = cargo
                self.score_length = score
                return True

        # Для поперечного.
        else:
            score = self._get_score_width(cargo)
            if abs(score) < abs(self.score_width):
                self.best_cargo = cargo
                self.score_width = score
                return True

        return False
    
    def _optimize(self, cargo: list):
        """ Процесс оптимизации delta. """

        for _ in range(self._n_iter):
            # Все перестановка имеющихся грузов.
            permutations_cargo = list(itertools.permutations(cargo))
    
            # Меняю очерёдность расположение каждого груза.
            for i in range(0, len(permutations_cargo)):
                copy_cargo = self._init_delta(permutations_cargo[i])
    
                # Эпохи для продольной оптимизации.
                flag_work_epoch = True
                while flag_work_epoch:
                    flag_work_epoch = False
    
                    # Смещаю каждый груз влево и вправо на step, затем смотрю метрики.
                    for k in range(len(copy_cargo)):
    
                        left_step_cargo = [{k:v for k, v in dct.items()} for dct in copy_cargo]
                        left_step_cargo[k]['delta_length'] -= self._step
                        
                        right_step_cargo = [{k:v for k, v in dct.items()} for dct in copy_cargo]
                        right_step_cargo[k]['delta_length'] += self._step
    
                        # Проверки на не наложение одного груза на другой.
                        # Нулевый элемент не выходит за пределы платформы слева.
                        if k == 0:
                            if left_step_cargo[k]['delta_length'] >= 0:
                                # проверка
                                if self._check_best_cargo(left_step_cargo, True):
                                    copy_cargo = self.best_cargo
                                    flag_work_epoch = True
                                    
                        # Последний элемент не выходит за пределы платформы справа.
                        elif k == len(copy_cargo) - 1:
                            if right_step_cargo[k]['delta_length'] + right_step_cargo[k]['length'] < L:
                                # проверка
                                if self._check_best_cargo(right_step_cargo, True):
                                    copy_cargo = self.best_cargo
                                    flag_work_epoch = True
                                
                        else:
                            # Проверка на не наложение правого груза на левый. 
                            if left_step_cargo[k]['delta_length'] > left_step_cargo[k-1]['delta_length'] + left_step_cargo[k-1]['length']:
                                if self._check_best_cargo(left_step_cargo, True):
                                    copy_cargo = self.best_cargo
                                    flag_work_epoch = True
    
                            # Проверка на не наложение левого груза на превый. 
                            if right_step_cargo[k]['delta_length'] + right_step_cargo[k]['length'] < right_step_cargo[k+1]['delta_length']:
                                if self._check_best_cargo(right_step_cargo, True):
                                    copy_cargo = self.best_cargo
                                    flag_work_epoch = True

                # Эпохи для поперечной оптимизации.
                flag_work_epoch = True
                while flag_work_epoch:
                    flag_work_epoch = False
    
                    # Смещаю каждый груз влево и вправо на step, затем смотрю метрики.
                    for k in range(len(copy_cargo)):
    
                        bottom_step_cargo = [{k:v for k, v in dct.items()} for dct in copy_cargo]
                        bottom_step_cargo[k]['delta_width'] -= self._step
                        
                        top_step_cargo = [{k:v for k, v in dct.items()} for dct in copy_cargo]
                        top_step_cargo[k]['delta_width'] += self._step
    
                        # Проверки на то, чтобы груз не выходил на пределы платформы по бокам.
                        if bottom_step_cargo[k]['delta_width'] >= 0:
                            # проверка
                            if self._check_best_cargo(bottom_step_cargo, False):
                                copy_cargo = self.best_cargo
                                flag_work_epoch = True
 
                        if top_step_cargo[k]['delta_width'] + top_step_cargo[k]['width'] < W:
                            # проверка
                            if self._check_best_cargo(top_step_cargo, False):
                                copy_cargo = self.best_cargo
                                flag_work_epoch = True
    



tmp = Optimize(cargo, step=1, n_iter=1000)
print(tmp.score_length, tmp.score_width)

0.0065309258534398396 1349.6799846331157


In [14]:
tmp.best_cargo

[{'length': 4100,
  'width': 1720,
  'height': 1150,
  'weight': 1.865,
  'delta_length': 145,
  'delta_width': 575.0},
 {'length': 3870,
  'width': 2890,
  'height': 1020,
  'weight': 4.085,
  'delta_length': 4531,
  'delta_width': -10.0},
 {'length': 1080,
  'width': 1580,
  'height': 390,
  'weight': 0.395,
  'delta_length': 8595,
  'delta_width': 645.0},
 {'length': 3650,
  'width': 3320,
  'height': 1500,
  'weight': 6.67,
  'delta_length': 9749,
  'delta_width': -225.0}]